In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = (
    SparkSession.builder
    .appName("Jupyter-on-Docker")
    .master("spark://spark-master:7077")
    .getOrCreate()
)

In [3]:
conf = spark._jsc.hadoopConfiguration()
print("S3 Endpoint:", conf.get("fs.s3a.endpoint"))
print("Access Key:", conf.get("fs.s3a.access.key"))
print("Secret Key:", conf.get("fs.s3a.secret.key"))


S3 Endpoint: http://minio:9000
Access Key: minio_access_key
Secret Key: minio_secret_key


In [6]:
configs = spark.sparkContext.getConf().getAll()
for k, v in configs:
    print(f"{k} = {v}")

spark.hadoop.fs.s3a.connection.ssl.enabled = false
spark.hadoop.fs.s3a.path.style.access = true
spark.sql.warehouse.dir = s3a://datalake/warehouse/
spark.hadoop.fs.s3a.aws.credentials.provider = org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider
spark.hadoop.fs.s3a.access.key = minio_access_key
spark.app.name = Jupyter-on-Docker
spark.driver.extraJavaOptions = -Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.act

In [5]:
src_csv = "s3a://datalake/warehouse/data.csv"
dst_parquet = "s3a://datalake/warehouse/parquet_file/orders"
df = spark.read.option("header", True).option("inferSchema", True).csv(src_csv)
df2 = df.dropna(subset=["CustomerID"]).dropDuplicates(["CustomerID"])
df2.write.mode("overwrite").parquet(dst_parquet)
spark.stop

Py4JJavaError: An error occurred while calling o46.csv.
: org.apache.spark.SparkException: Cannot find catalog plugin class for catalog 'spark_catalog': org.apache.spark.sql.delta.catalog.DeltaCatalog.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.catalogPluginClassNotFoundForCatalogError(QueryExecutionErrors.scala:1925)
	at org.apache.spark.sql.connector.catalog.Catalogs$.load(Catalogs.scala:70)
	at org.apache.spark.sql.connector.catalog.CatalogManager.loadV2SessionCatalog(CatalogManager.scala:67)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$v2SessionCatalog$2(CatalogManager.scala:86)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$v2SessionCatalog$1(CatalogManager.scala:86)
	at scala.Option.map(Option.scala:230)
	at org.apache.spark.sql.connector.catalog.CatalogManager.v2SessionCatalog(CatalogManager.scala:85)
	at org.apache.spark.sql.connector.catalog.CatalogManage

In [9]:
spark

In [6]:
import delta
print("Delta version:", delta.__version__)

AttributeError: module 'delta' has no attribute '__version__'

In [6]:
print(spark.conf.get("spark.jars.packages"))


Py4JJavaError: An error occurred while calling o51.get.
: org.apache.spark.SparkNoSuchElementException: [SQL_CONF_NOT_FOUND] The SQL config "spark.jars.packages" cannot be found. Please verify that the config exists.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.sqlConfigNotFoundError(QueryExecutionErrors.scala:1984)
	at org.apache.spark.sql.internal.SQLConf.$anonfun$getConfString$3(SQLConf.scala:5234)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.internal.SQLConf.getConfString(SQLConf.scala:5234)
	at org.apache.spark.sql.RuntimeConfig.get(RuntimeConfig.scala:81)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:568)
	at py4j.reflection.MethodInvoker.invoke(M

In [4]:
print("Delta in jars:", "delta" in str(spark.sparkContext._conf.get("spark.jars")))


Delta in jars: True
